In [106]:
import re
import polars as pl

In [107]:
# Function to parse the log file
def parse_log_file(log_file_path):
    user_list = []
    token_list = []
    
    with open(log_file_path, "r") as f:
        for line in f:
            # Check if the line contains the word 'confirm'
            if 'confirm' in line:
                # Extract user and token using regular expressions
                user_match = re.search(r'trader (\d+): confirm', line)
                token_match = re.search(r'token (\w+)\.', line)
                
                if user_match and token_match:
                    user = user_match.group(1)
                    token = token_match.group(1)
                    
                    user_list.append(user)
                    token_list.append(token)

    # Create a Polars DataFrame
    df = pl.DataFrame(
        {
            "user": user_list,
            "token": token_list
        }
    )

    return df.unique()


In [108]:
# Path to your log file
log_file_path = "log_hft.txt"

# Parse the log file and create a Polars DataFrame
df = parse_log_file(log_file_path)

# Show the DataFrame
print(df)


shape: (408, 2)
┌──────┬────────────────┐
│ user ┆ token          │
│ ---  ┆ ---            │
│ str  ┆ str            │
╞══════╪════════════════╡
│ 3421 ┆ AAAAB342100097 │
│ 3422 ┆ BBBBS342200193 │
│ 3421 ┆ AAAAB342100169 │
│ 1    ┆ INVES000100136 │
│ …    ┆ …              │
│ 1    ┆ INVES000100171 │
│ 1    ┆ INVES000100133 │
│ 3421 ┆ AAAAB342100359 │
│ 1    ┆ INVEB000100010 │
└──────┴────────────────┘


In [109]:
# Count unique users per token
users_by_token = df.groupby("token").agg(
    pl.col("user").n_unique().alias("unique_users")
)

In [110]:
users_by_token.filter(pl.col("unique_users") > 1)

token,unique_users
str,u32


In [111]:
# Count unique tokens per user
tokens_by_user = df.groupby("user").agg(
    pl.col("token").n_unique().alias("unique_tokens")
)

print(tokens_by_user)

shape: (3, 2)
┌──────┬───────────────┐
│ user ┆ unique_tokens │
│ ---  ┆ ---           │
│ str  ┆ u32           │
╞══════╪═══════════════╡
│ 3421 ┆ 103           │
│ 3422 ┆ 75            │
│ 1    ┆ 230           │
└──────┴───────────────┘


In [112]:
def read_token_crossing_log(file_path):
    buy_tokens = []
    sell_tokens = []
    crossing_prices = []
    volumes = []
    
    # Regex pattern to extract buy token, sell token, crossing price, and volume
    pattern = r"Orders \(b'(.*?)', b'(.*?)'\) crossed at price (\d+), volume (\d+)"
    
    # Read log file line-by-line and extract relevant information
    with open(file_path, 'r') as f:
        for line in f:
            if 'root.process_cross' in line:
                match = re.search(pattern, line)
                if match:
                    buy_token, sell_token, crossing_price, volume = match.groups()
                    buy_tokens.append(buy_token)
                    sell_tokens.append(sell_token)
                    crossing_prices.append(int(crossing_price))
                    volumes.append(int(volume))
                    
    # Create a DataFrame using the extracted information
    df_crossing = pl.DataFrame({
        'buy_token': buy_tokens,
        'sell_token': sell_tokens,
        'crossing_price': crossing_prices,
        'volume': volumes
    })
    
    return df_crossing.unique()

In [113]:
# Example usage
file_path = 'log_exchange_server.txt'  # Replace with the actual path to your log file
df_crossing = read_token_crossing_log(file_path)
print(df_crossing)

shape: (109, 4)
┌────────────────┬────────────────┬────────────────┬────────┐
│ buy_token      ┆ sell_token     ┆ crossing_price ┆ volume │
│ ---            ┆ ---            ┆ ---            ┆ ---    │
│ str            ┆ str            ┆ i64            ┆ i64    │
╞════════════════╪════════════════╪════════════════╪════════╡
│ AAAAS342100019 ┆ INVEB000100015 ┆ 1690000        ┆ 1      │
│ INVES000100044 ┆ AAAAB342100063 ┆ 1160000        ┆ 1      │
│ AAAAB342100280 ┆ INVES000100171 ┆ 1140000        ┆ 1      │
│ AAAAS342100368 ┆ INVEB000100194 ┆ 1080000        ┆ 1      │
│ …              ┆ …              ┆ …              ┆ …      │
│ INVES000100032 ┆ BBBBB342200023 ┆ 1200000        ┆ 1      │
│ INVES000100094 ┆ INVEB000100087 ┆ 1150000        ┆ 1      │
│ AAAAB342100093 ┆ INVES000100055 ┆ 1150000        ┆ 1      │
│ INVES000100093 ┆ INVEB000100085 ┆ 1170000        ┆ 1      │
└────────────────┴────────────────┴────────────────┴────────┘


In [114]:
# Joining to get buyer information
final_df = (
    df_crossing
    .join(df, left_on="buy_token", right_on="token", how="left")
    .rename({"user": "buyer"})
    .join(df, left_on="sell_token", right_on="token", how="left")
    .rename({"user": "seller"})
    )


In [126]:
print(final_df.filter(
    (pl.col("buyer") is None) |
    (pl.col("seller") is None)

))

shape: (0, 6)
┌───────────┬────────────┬────────────────┬────────┬───────┬────────┐
│ buy_token ┆ sell_token ┆ crossing_price ┆ volume ┆ buyer ┆ seller │
│ ---       ┆ ---        ┆ ---            ┆ ---    ┆ ---   ┆ ---    │
│ str       ┆ str        ┆ i64            ┆ i64    ┆ str   ┆ str    │
╞═══════════╪════════════╪════════════════╪════════╪═══════╪════════╡
└───────────┴────────────┴────────────────┴────────┴───────┴────────┘


In [115]:
print(final_df)


shape: (109, 6)
┌────────────────┬────────────────┬────────────────┬────────┬───────┬────────┐
│ buy_token      ┆ sell_token     ┆ crossing_price ┆ volume ┆ buyer ┆ seller │
│ ---            ┆ ---            ┆ ---            ┆ ---    ┆ ---   ┆ ---    │
│ str            ┆ str            ┆ i64            ┆ i64    ┆ str   ┆ str    │
╞════════════════╪════════════════╪════════════════╪════════╪═══════╪════════╡
│ AAAAS342100019 ┆ INVEB000100015 ┆ 1690000        ┆ 1      ┆ 3421  ┆ 1      │
│ INVES000100044 ┆ AAAAB342100063 ┆ 1160000        ┆ 1      ┆ 1     ┆ 3421   │
│ AAAAB342100280 ┆ INVES000100171 ┆ 1140000        ┆ 1      ┆ 3421  ┆ 1      │
│ AAAAS342100368 ┆ INVEB000100194 ┆ 1080000        ┆ 1      ┆ 3421  ┆ 1      │
│ …              ┆ …              ┆ …              ┆ …      ┆ …     ┆ …      │
│ INVES000100032 ┆ BBBBB342200023 ┆ 1200000        ┆ 1      ┆ 1     ┆ 3422   │
│ INVES000100094 ┆ INVEB000100087 ┆ 1150000        ┆ 1      ┆ 1     ┆ 1      │
│ AAAAB342100093 ┆ INVES000100055 ┆ 

In [116]:
self_crossing_df = final_df.filter(
    (pl.col("buyer") == pl.col("seller")) &
    (pl.col("buyer") != "1")
)


In [125]:
print(self_crossing_df)

shape: (1, 6)
┌────────────────┬────────────────┬────────────────┬────────┬───────┬────────┐
│ buy_token      ┆ sell_token     ┆ crossing_price ┆ volume ┆ buyer ┆ seller │
│ ---            ┆ ---            ┆ ---            ┆ ---    ┆ ---   ┆ ---    │
│ str            ┆ str            ┆ i64            ┆ i64    ┆ str   ┆ str    │
╞════════════════╪════════════════╪════════════════╪════════╪═══════╪════════╡
│ AAAAS342100318 ┆ AAAAB342100313 ┆ 1160000        ┆ 1      ┆ 3421  ┆ 3421   │
└────────────────┴────────────────┴────────────────┴────────┴───────┴────────┘


In [118]:
self_crossing_df.write_csv("self_crossing.csv")

In [119]:
filtered_df_missing_info = final_df.filter(
    pl.col("buyer").is_null() | pl.col("seller").is_null()
)

In [120]:
filtered_df_missing_info

buy_token,sell_token,crossing_price,volume,buyer,seller
str,str,i64,i64,str,str
